In [1]:
# X
# tijdstip: dag?
# Verbruik (kW, t)​ -- kW in de tijd
# 	Historische Verbruik​
# 	Temperatuur
# 	Weekdag
# 	Seizoen (vakantie)​
# 	Laadpalen

# Opwekking (kW, t)​ -- kW in de tijd
# 	Zonne-energie​
# 		Zonuren​
# 		% wolken​
# 		Wattpiek​
# 	Windenergie​
# 		Windrichting​
# 		Windsnelheid​
# 		Opgesteld vermogen​

# Y


# over welke periode gaat het dataframe, 3 jaar, 10 jaar?

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [3]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [3]:
# variables used in script
data_processed_location = '../data/processed'

In [ ]:
if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [5]:
os.getcwd()

'C:\\Users\\MarcelKoolwijk\\Documents\\SQL\\jads_enexis\\data\\processed'

In [6]:
# meteo data inlezen
df_meteo = pd.read_hdf('meteo_data.h5')

In [7]:
# decentrale_opwekkingsgegevens gegevens inlezen
df_opwek = pd.read_hdf('decentrale_opwekkingsgegevens_data.h5')

In [8]:
# kleinverbruikgegevens gegevens inlezen
df_verbruik = pd.read_hdf('kleinverbruikgegevens_data.h5')

In [9]:
df_meteo.head()

,STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,...,NG,UG,UX,UXH,UN,UNH,EV24,LOC,LAT,LNG
0,209,20010130,,,,,,,,,...,,,,,,,,IJmond,52.496506,4.658565
1,209,20010131,,,,,,,,,...,,,,,,,,IJmond,52.496506,4.658565
2,209,20010201,236,13,24,50,8,10,1,90,...,,,,,,,,IJmond,52.496506,4.658565
3,209,20010202,150,46,58,80,9,10,3,100,...,,,,,,,,IJmond,52.496506,4.658565
4,209,20010203,106,65,66,90,8,50,1,120,...,,,,,,,,IJmond,52.496506,4.658565


In [10]:
df_opwek.head()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,PC4
0,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Centrum,30000,1480,155,507,9901
1,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-West,30001,1331,479,1685,9903
2,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Oost,30002,2826,439,1616,9902
3,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,30007,230,97,405,9902
4,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,30008,102,21,61,9902


In [11]:
df_verbruik.head()

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR,PC4,SJV_TOTAAL
0,Enexis B.V.,ENEXIS,Sasdijk,4251AB,4251AB,WERKENDAM,NL,ELK,KVB,100.0,16,100,Onbekend,Onbekend,4282.0,25.0,0.0,2010,4251,69.0
1,Enexis B.V.,ENEXIS,Sasdijk,4251AC,4251AC,WERKENDAM,NL,ELK,KVB,100.0,11,100,Onbekend,Onbekend,5113.0,10.0,0.0,2010,4251,56.0
2,Enexis B.V.,ENEXIS,Sasdijk,4251AD,4251AD,WERKENDAM,NL,ELK,KVB,100.0,30,100,Onbekend,Onbekend,4809.0,34.0,0.0,2010,4251,144.0
3,Enexis B.V.,ENEXIS,Nieuweweg,4251AE,4251AG,WERKENDAM,NL,ELK,KVB,100.0,21,100,Onbekend,Onbekend,5015.0,44.0,0.0,2010,4251,105.0
4,Enexis B.V.,ENEXIS,Koppenhof,4251AH,4251AH,WERKENDAM,NL,ELK,KVB,100.0,12,100,Onbekend,Onbekend,3074.0,22.0,0.0,2010,4251,37.0


In [12]:
# alles group by dag?
# wat als we maar 1 waarde hebben per jaar / half jaar?

In [18]:
# sample verbruik data genereren obv jaar totaal
#df_dates = pd.DataFrame(pd.date_range(start='1/1/2010', end='31/12/2020'), columns=['datum']).sort_values(by='datum', ascending=False )
df_dates = pd.DataFrame(pd.date_range(start='1/1/2018', end='31/12/2020'), columns=['datum']).sort_values(by='datum', ascending=False )

In [19]:
df_dates['jaar'] = df_dates['datum'].dt.year
df_dates['maand'] = df_dates['datum'].dt.to_period('M')
df_dates['maand_van_jaar'] = df_dates['datum'].dt.month
df_dates['dag_van_week'] = df_dates['datum'].dt.dayofweek

In [20]:
df_dates.head()

,datum,jaar,maand,maand_van_jaar,dag_van_week
1095,2020-12-31,2020,2020-12,12,3
1094,2020-12-30,2020,2020-12,12,2
1093,2020-12-29,2020,2020-12,12,1
1092,2020-12-28,2020,2020-12,12,0
1091,2020-12-27,2020,2020-12,12,6


In [21]:
# join met df_verbruik
# join on jaar
# SJV_TOTAAL en POSTCODE_VAN en POSTCODE_TOT

In [22]:
#df_dates.info()
#df_verbruik.info()

In [23]:
df_verbruik['JAAR'] = df_verbruik['JAAR'].astype(float)

In [24]:
# selecteer kolommen van df_verbruik
df_verbruik_sel = df_verbruik.loc[:, ['JAAR', 'POSTCODE_VAN', 'POSTCODE_TOT', 'SJV_TOTAAL']]

In [25]:
#print(df_dates.shape)
#print(df_verbruik_sel.shape)

In [26]:
# join df_dates met df_verbruik
df_dates = pd.merge(df_dates, df_verbruik_sel, left_on='jaar', right_on='JAAR', how='left', suffixes=('','_2'))

In [27]:
df_dates.head()

,datum,jaar,maand,maand_van_jaar,dag_van_week,JAAR,POSTCODE_VAN,POSTCODE_TOT,SJV_TOTAAL
0,2020-12-31,2020,2020-12,12,3,2020.0,4251 AA,4251 AA,232.0
1,2020-12-31,2020,2020-12,12,3,2020.0,4251 AB,4251 AB,68.0
2,2020-12-31,2020,2020-12,12,3,2020.0,4251 AC,4251 AC,28.0
3,2020-12-31,2020,2020-12,12,3,2020.0,4251 AD,4251 AD,129.0
4,2020-12-31,2020,2020-12,12,3,2020.0,4251 AE,4251 AG,105.0


In [28]:
df_dates.shape

(136498367, 9)

In [31]:
factoren_maand = {
    '1': 1.13,
    '2': 1.03,
    '3': 1.08,
    '4': 0.93,
    '5': 0.93,
    '6': 0.93,
    '7': 0.93,
    '8': 0.93,
    '9': 0.93,
    '10': 1.03,
    '11': 1.03,
    '12': 1.08
}

In [37]:
#df_dates['factor_maand']  = 1/365
df_dates['factor_maand'] = df_dates['maand_van_jaar'].astype(str).map(factoren_maand)

In [39]:
df_dates['factor'] = (1/365) * df_dates['factor_maand']

In [41]:
df_dates['dag_verbruik_kw'] = df_dates['factor'] * df_dates['SJV_TOTAAL']

In [42]:
df_dates.head()

,datum,jaar,maand,maand_van_jaar,dag_van_week,JAAR,POSTCODE_VAN,POSTCODE_TOT,SJV_TOTAAL,factor_maand,factor,dag_verbruik_kw
0,2020-12-31,2020,2020-12,12,3,2020.0,4251 AA,4251 AA,232.0,1.08,0.002959,0.686466
1,2020-12-31,2020,2020-12,12,3,2020.0,4251 AB,4251 AB,68.0,1.08,0.002959,0.201205
2,2020-12-31,2020,2020-12,12,3,2020.0,4251 AC,4251 AC,28.0,1.08,0.002959,0.082849
3,2020-12-31,2020,2020-12,12,3,2020.0,4251 AD,4251 AD,129.0,1.08,0.002959,0.381699
4,2020-12-31,2020,2020-12,12,3,2020.0,4251 AE,4251 AG,105.0,1.08,0.002959,0.310685


In [36]:
df_dates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136498367 entries, 0 to 136498366
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   datum           datetime64[ns]
 1   jaar            int64         
 2   maand           period[M]     
 3   maand_van_jaar  int64         
 4   dag_van_week    int64         
 5   JAAR            float64       
 6   POSTCODE_VAN    object        
 7   POSTCODE_TOT    object        
 8   SJV_TOTAAL      float64       
 9   factor_maand    float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(2), period[M](1)
memory usage: 11.2+ GB
